<a href="https://colab.research.google.com/github/HahyunKang/Datamining_Project/blob/main/BookRecommenderByMovie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/AJOU 23-2/Data Mining/Team Project

/content/drive/MyDrive/AJOU 23-2/Data Mining/Team Project


In [3]:
ls

book_data_with_keywords.csv  books_list.csv  Movie.ipynb          texkRank.ipynb
book.ipynb                   keywords.csv    movies_metadata.csv


In [4]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib inline

# Movies

In [5]:
movies = pd.read_csv("movies_metadata.csv")

movies.head()

<ipython-input-5-7cc4136ec243>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("movies_metadata.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [6]:
movies['genres'][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [7]:
movies['genres'] = movies['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [8]:
keywords = pd.read_csv('keywords.csv')

In [9]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [11]:
ratings = pd.read_csv('ratings.csv')

In [12]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [13]:
average_ratings = ratings.groupby('movieId')['rating'].mean().reset_index()
average_ratings.rename(columns={'rating': 'average_rating'}, inplace=True)

In [14]:
average_ratings = average_ratings.sort_values('movieId')

average_ratings.set_index('movieId', inplace=True)

average_ratings.rename(columns={'movieId': 'id'}, inplace=True)

In [15]:
average_ratings.head()

,average_rating
movieId,
1,3.888157
2,3.236953
3,3.175550
4,2.875713
5,3.079565


In [16]:
movies['id'] = pd.to_numeric(movies['id'], errors='coerce')

movies = movies.dropna(subset=['id'])

In [17]:
movies['id'] = movies['id'].astype('int')
keywords['id'] = keywords['id'].astype('int')

movies = movies.merge(keywords, on='id')
movies = movies.merge(average_ratings, left_on='id', right_index=True)

movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,keywords,average_rating
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",3.598930
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'id': 10090, 'name': 'board game'}, {'id': 1...",3.760163
5,False,NaN,60000000,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,"[{'id': 642, 'name': 'robbery'}, {'id': 703, '...",3.905544
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[Adventure, Action, Thriller]",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0,"[{'id': 701, 'name': 'cuba'}, {'id': 769, 'nam...",2.740334
14,False,NaN,98000000,"[Action, Adventure]",NaN,1408,tt0112760,en,Cutthroat Island,"Morgan Adams and her slave, William Shaw, are ...",...,119.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Course Has Been Set. There Is No Turning B...,Cutthroat Island,False,5.7,137.0,"[{'id': 911, 'name': 'exotic island'}, {'id': ...",3.710181


In [18]:
movies["keywords"][0]

"[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]"

In [19]:
movies["keywords"] = movies["keywords"].apply(literal_eval)

In [20]:
def generate_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names) > 10:
            names = names[:10]
        return names
    return []

movies['keywords'] = movies['keywords'].apply(generate_list)
movies['genres'] = movies['genres'].apply(lambda x: x[:10])

movies[['title', 'keywords', 'genres', 'average_rating']].head()

,title,keywords,genres,average_rating
0,Toy Story,"[jealousy, toy, boy, friendship, friends, riva...","[Animation, Comedy, Family]",3.598930
1,Jumanji,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]",3.760163
5,Heat,"[robbery, detective, bank, obsession, chase, s...","[Action, Crime, Drama, Thriller]",3.905544
9,GoldenEye,"[cuba, falsely accused, secret identity, compu...","[Adventure, Action, Thriller]",2.740334
14,Cutthroat Island,"[exotic island, treasure, map, ship, scalp, pi...","[Action, Adventure]",3.710181


In [22]:
def normalize_text(x):
  if isinstance(x, list):
        return [elem.lower().replace(' ', '') for elem in x]
  return x.lower().replace(' ', '') if isinstance(x, str) else ''

for feature in ['genres', 'keywords']:
    movies[feature] = movies[feature].apply(normalize_text)

In [23]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,keywords,average_rating
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[animation, comedy, family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[jealousy, toy, boy, friendship, friends, riva...",3.598930
1,False,NaN,65000000,"[adventure, fantasy, family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[boardgame, disappearance, basedonchildren'sbo...",3.760163
5,False,NaN,60000000,"[action, crime, drama, thriller]",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,"[robbery, detective, bank, obsession, chase, s...",3.905544
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[adventure, action, thriller]",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0,"[cuba, falselyaccused, secretidentity, compute...",2.740334
14,False,NaN,98000000,"[action, adventure]",NaN,1408,tt0112760,en,Cutthroat Island,"Morgan Adams and her slave, William Shaw, are ...",...,119.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Course Has Been Set. There Is No Turning B...,Cutthroat Island,False,5.7,137.0,"[exoticisland, treasure, map, ship, scalp, pir...",3.710181


In [24]:
def movie_features(x):
    return (x["title"] + " " +
            " ".join(x['genres']) + " " +
            x['overview'] + " " +
            " ".join(x['keywords']) + " " +
            str(x['average_rating']))

movies['overview'] = movies['overview'].fillna('')
movies['title'] = movies['title'].fillna('')
movies['average_rating'] = movies['average_rating'].fillna(0)

movies['movie_features'] = movies.apply(movie_features, axis=1)

In [25]:
movies.loc[movies['title']=="The Matrix",'movie_features'].values

array(['The Matrix action sciencefiction Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth. savingtheworld artificialintelligence manvsmachine philosophy prophecy martialarts selfsacrifice fight insurgence virtualreality 2.9557877813504825'],
      dtype=object)

# Books

In [26]:
books = pd.read_csv("book_data_with_keywords.csv", index_col=0)

In [28]:
books.head()

,title,author,rating,voters,price,currency,description,publisher,page_count,generes,ISBN,language,published_date,keywords
id,,,,,,,,,,,,,,
0,Attack on Titan: Volume 13,Hajime Isayama,4.6,428,43.28,SAR,NO SAFE PLACE LEFT At great cost to the Garris...,Kodansha Comics,192.0,none,9.78E+12,English,31-Jul-14,"['survey', 'eren', ""titans'"", 'titans', ""doesn..."
1,Antiques Roadkill: A Trash 'n' Treasures Mystery,Barbara Allan,3.3,23,26.15,SAR,Determined to make a new start in her quaint h...,Kensington Publishing Corp.,288.0,"Fiction , Mystery &amp, Detective , Cozy , Gen...",9.78E+12,English,01-Jul-07,"['serenity', 'becomes', 'number', ""victim's"", ..."
2,The Art of Super Mario Odyssey,Nintendo,3.9,9,133.85,SAR,Take a globetrotting journey all over the worl...,Dark Horse Comics,368.0,"Games &amp, Activities , Video &amp, Electronic",9.78E+12,English,05-Nov-19,"['world', 'mario', 'game', 'art', 'captivating']"
3,Getting Away Is Deadly: An Ellie Avery Mystery,Sara Rosett,4.0,10,26.15,SAR,"With swollen feet and swelling belly, pregnant...",Kensington Publishing Corp.,320.0,none,9.78E+12,English,01-Mar-09,"['keeps', 'hiding', 'pack', 'belly,', 'sights']"
4,"The Painted Man (The Demon Cycle, Book 1)",Peter V. Brett,4.5,577,28.54,SAR,The stunning debut fantasy novel from author P...,HarperCollins UK,544.0,"Fiction , Fantasy , Dark Fantasy",9.78E+12,English,08-Jan-09,"['demons', 'darkness', 'world', 'women', 'wards']"


In [29]:
books['keywords'][1]

'[\'serenity\', \'becomes\', \'number\', "victim\'s", \'seniors\']'

In [42]:
len(books[books['generes'] == 'none'])

527

In [31]:
len(books)

1299

In [35]:
books['keywords'] = books['keywords'].apply(lambda x: literal_eval(x) if literal_eval(x) else np.nan)

In [36]:
books.isnull().sum()

title              0
author             0
rating            75
voters            75
price              0
currency           0
description        3
publisher          0
page_count         0
generes            0
ISBN               0
language           0
published_date     0
keywords           0
dtype: int64

In [37]:
books = books[
    (books['title'].notnull()) |
    (books['description'].notnull()) |
    (books['generes'].notnull()) |
    (books['keywords'].notnull())
]
books = books.fillna('')

In [43]:
books.loc[books['generes'] == 'none', 'generes'] = ''

In [44]:
books.head()

,title,author,rating,voters,price,currency,description,publisher,page_count,generes,ISBN,language,published_date,keywords
id,,,,,,,,,,,,,,
0,Attack on Titan: Volume 13,Hajime Isayama,4.6,428,43.28,SAR,NO SAFE PLACE LEFT At great cost to the Garris...,Kodansha Comics,192.0,,9.78E+12,English,31-Jul-14,"[survey, eren, titans', titans, doesn't]"
1,Antiques Roadkill: A Trash 'n' Treasures Mystery,Barbara Allan,3.3,23,26.15,SAR,Determined to make a new start in her quaint h...,Kensington Publishing Corp.,288.0,"Fiction , Mystery &amp, Detective , Cozy , Gen...",9.78E+12,English,01-Jul-07,"[serenity, becomes, number, victim's, seniors]"
2,The Art of Super Mario Odyssey,Nintendo,3.9,9,133.85,SAR,Take a globetrotting journey all over the worl...,Dark Horse Comics,368.0,"Games &amp, Activities , Video &amp, Electronic",9.78E+12,English,05-Nov-19,"[world, mario, game, art, captivating]"
3,Getting Away Is Deadly: An Ellie Avery Mystery,Sara Rosett,4.0,10,26.15,SAR,"With swollen feet and swelling belly, pregnant...",Kensington Publishing Corp.,320.0,,9.78E+12,English,01-Mar-09,"[keeps, hiding, pack, belly,, sights]"
4,"The Painted Man (The Demon Cycle, Book 1)",Peter V. Brett,4.5,577,28.54,SAR,The stunning debut fantasy novel from author P...,HarperCollins UK,544.0,"Fiction , Fantasy , Dark Fantasy",9.78E+12,English,08-Jan-09,"[demons, darkness, world, women, wards]"


In [54]:
books.dtypes

title              object
author             object
rating             object
voters             object
price             float64
currency           object
description        object
publisher          object
page_count        float64
generes            object
ISBN               object
language           object
published_date     object
keywords           object
dtype: object

In [55]:
def book_features(x):
    return (x["title"] + " " +
            x["generes"] + " " +
            x["description"] + " " +
            " ".join(x['keywords']) + " " +
            str(x["rating"]))

In [59]:
books['books_features'] = books.apply(book_features, axis=1)

In [60]:
books['books_features'].iloc[0]

"Attack on Titan: Volume 13  NO SAFE PLACE LEFT At great cost to the Garrison and the Survey Corps, Commander Erwin has managed to recover Eren from the Titans who tried to carry him off. But during the battle, Eren manifested yet another power he doesn't understand. As Eren and Krista find new enemies, the Survey Corps faces threats from both inside and outside the walls. And what will happen to Ymir, now that she has decided to make herself the Titans' prize? survey eren titans' titans doesn't 4.6"

# Vectorizing Data

In [63]:
combined_features = pd.concat([movies['movie_features'],books['books_features']],ignore_index=True)

In [69]:
# Count Vectorizer 설정 및 학습
count = CountVectorizer(stop_words="english")
count.fit(combined_features)

CountVectorizer(stop_words='english')

In [70]:
# 영화와 책의 Count Vectorized matrix 생성
movies_matrix = count.transform(movies['movie_features'])
books_matrix = count.transform(books['books_features'])

In [71]:
books_matrix.shape, movies_matrix.shape

((1299, 41561), (7677, 41561))

In [72]:
# Cosine Similarity 계산
cosine_sim = cosine_similarity(movies_matrix, books_matrix)

In [75]:
# 영화의 인덱스를 기준으로 Series 생성
movies = movies.reset_index()
indices = pd.Series(movies.index, index=movies['title'].apply(lambda x: x.lower() if x is not np.nan else "")).drop_duplicates()

In [76]:
movies.head()

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,keywords,average_rating,movie_features
0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[animation, comedy, family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[jealousy, toy, boy, friendship, friends, riva...",3.598930,Toy Story animation comedy family Led by Woody...
1,1,False,NaN,65000000,"[adventure, fantasy, family]",NaN,8844,tt0113497,en,Jumanji,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[boardgame, disappearance, basedonchildren'sbo...",3.760163,Jumanji adventure fantasy family When siblings...
2,5,False,NaN,60000000,"[action, crime, drama, thriller]",NaN,949,tt0113277,en,Heat,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,"[robbery, detective, bank, obsession, chase, s...",3.905544,Heat action crime drama thriller Obsessive mas...
3,9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[adventure, action, thriller]",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0,"[cuba, falselyaccused, secretidentity, compute...",2.740334,GoldenEye adventure action thriller James Bond...
4,14,False,NaN,98000000,"[action, adventure]",NaN,1408,tt0112760,en,Cutthroat Island,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Course Has Been Set. There Is No Turning B...,Cutthroat Island,False,5.7,137.0,"[exoticisland, treasure, map, ship, scalp, pir...",3.710181,Cutthroat Island action adventure Morgan Adams...


In [78]:
# # Content-based 추천 시스템 함수 정의
# def book_recommender_by_movie(title):
#     idx = indices[title.lower()]
#     similarity_scores = list(enumerate(cosine_sim[idx]))
#     similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
#     similarity_scores = similarity_scores[:10]
#     recommended_book_indices = [i[0] for i in similarity_scores]
#     return books.iloc[recommended_book_indices].copy()  # 복사해서 반환


In [92]:
# Content-based 추천 시스템 함수 정의
def book_recommender_by_movie(title):
    idx = indices[title.lower()]
    similarity_scores = list(enumerate(cosine_sim[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    recommended_book_indices = []
    recommended_books = set()  # 중복 제거를 위한 집합

    for i, score in similarity_scores:
        book_index = i
        book_title = books.iloc[book_index]['title']

        if book_title not in recommended_books:  # 중복되지 않는 경우만 추가
            recommended_books.add(book_title)
            recommended_book_indices.append(book_index)

            if len(recommended_book_indices) == 10:  # 10개까지 추천
                break

    return books.iloc[recommended_book_indices].copy()  # 복사해서 반환


In [96]:
from IPython.display import display

def print_book_info(books_df):
    book_info = books_df[['title', 'author', 'generes']]
    display(book_info)

In [97]:
def get_movie_preference():
    print("영화 취향을 입력하세요:")
    movie_preference = input()

    recommended_books = book_recommender_by_movie(movie_preference)

    print("이런 책을 추천합니다:")
    print_book_info(recommended_books)  # 추천된 책 출력

In [98]:
# 영화 취향 입력 받기
get_movie_preference()

영화 취향을 입력하세요:
toy story
이런 책을 추천합니다:


,title,author,generes
id,,,
201,Battle Sight Zero,Gerald Seymour,
949,Tall Tales and Wee Stories: The Best of Billy ...,Billy Connolly,"Performing Arts , Television , Genres , Comedy"
783,How to Win Friends and Influence People,Dale Carnegie,"Family &amp, Relationships , Friendship"
1213,Legacy,Shannon Messenger,"Juvenile Fiction , Social Themes , Friendship"
740,Boy Erased: A Memoir,Garrard Conley,
85,Heart and Brain: An Awkward Yeti Collection,The Awkward Yeti,
712,Gangster: A Novel,Lorenzo Carcaterra,
702,10 Ways to be annoying,James Fries,"Family &amp, Relationships , Friendship"
590,It,Stephen King,
